# Hotel Cancellation Machine Learning Modell

In [4]:
#installieren der relevanten libraries
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#importieren der relevanten libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix

%matplotlib inline

## 1. Load and inspect dataset

The Kaggle dataset on hotel reservations provides detailed information about bookings made by customers. It offers valuable insights for analyzing booking patterns, demand forecasting, and customer behavior in the hospitality industry. With a rich collection of data, it serves as a valuable resource for researchers and data scientists working on hotel management and related fields.

In [ ]:
#laden des main dataframes und Darstellung der ersten 5 Zeilen
csv_url = "https://raw.githubusercontent.com/Flitschi7/stornierungsvorhersage-im-hotelgewerbe/main/hotel_reservations.csv"
main_df = pd.read_csv(csv_url)
main_df.head()


In [ ]:
#Dataframe mit den Spaltennamen und den Datentypen der Spalten erstellen
main_df.describe(include='all')

In [ ]:
#überprüfen wie die Verteilung der stornierten Buchungen ist
booking_status_count = main_df.groupby('booking_status')['Booking_ID'].count().reset_index()
def func(pct):
  return "{:1.1f}%".format(pct)
plt.pie(booking_status_count.Booking_ID.values, labels=booking_status_count.booking_status.values, autopct=lambda pct: func(pct), shadow=True)
plt.title('Gesamte Buchungen')
plt.show()


Da die Verteilung nicht 1/10 sondern ca 3/10 ist. Wird erstmal keine Art der Datengleichstellung betrachtet.

# 2. Data cleaning

In [ ]:
#Duplikate in den Daten finden
main_df[main_df.duplicated(keep=False)] 

In [ ]:
#Duplikate in den IDs finden
main_df['Booking_ID'][main_df['Booking_ID'].duplicated(keep=False)] 

In [ ]:
#Nullwerte in den Daten finden
main_df.isnull().sum()

In [ ]:
#ID Spalte löschen
main_df = main_df.drop(['Booking_ID'], axis = 1)


In [ ]:
#arrival year und date sind keine relevanten features
main_df = main_df.drop(['arrival_year','arrival_date'], axis = 1)
#arrival month kann noch als feature genutzt werden da eventuell saisonale Schwankungen auftreten

In [ ]:
main_df.describe(include='all')

In [ ]:
#ändern des Datentyps der Spalte booking_status
main_df['booking_status'] = main_df['booking_status'].map({'Canceled': 0, 'Not_Canceled': 1})


In [ ]:
main_df.describe(include='all')

In [ ]:
clean_df =main_df.copy()

# 3. Data Understanding

## 3.1 checking numeric Data

In [ ]:
numeric_features = clean_df.select_dtypes(include=[np.number])
numeric_features.dtypes


required_car_parking_space und repeated_guest sind kategoriale daten, da es nur ja (1) oder nein (0) gibt.

### 3.1.1 Vizualisation of the data

In [ ]:
features = ['no_of_adults', 'arrival_month', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled','avg_price_per_room','no_of_special_requests']

fig, axs = plt.subplots(4, 3, figsize=(20, 15))
axs = np.ravel(axs)

for i, feature in enumerate(features):
    sns.histplot(numeric_features[feature], kde=True, stat='density', kde_kws=dict(cut=3), bins=100, color='gray', alpha=0.4, linewidth=0.25, ax=axs[i])
    axs[i].set_title(f'Verteilung von {feature}')
    axs[i].set_xlabel(feature)
    axs[i].set_ylabel('Dichte')

plt.tight_layout()
plt.show()


In [ ]:
for feature in features:
    distribution = numeric_features[feature].value_counts().reset_index()
    distribution.columns = [feature, 'frequency']
    distribution = distribution.sort_values(feature)
    
    print(f"Verteilung von '{feature}':")
    print(distribution)
    print()

## 3.1.2 Interpretation of the data
#### Verteilung von 'no_of_adults':

Die Mehrheit der Buchungen hat 2 Erwachsene.
Es gibt jedoch auch einige Buchungen mit 1, 3 und 4 Erwachsenen.
Es gibt 139 Buchungen, bei denen keine Erwachsenen angegeben sind, was möglicherweise auf Fehler oder ungewöhnliche Situationen hinweist.

#### Verteilung von arrival_month

Die Mehrheit der Besuche erfolgt im Oktober, das lässt eventuell darauf schließen dass hier eine Saisonalität vorliegt

#### Verteilung von 'no_of_children':

Die Mehrheit der Buchungen hat keine Kinder.
Es gibt jedoch auch Buchungen mit 1, 2, 3, 4, 5 und sogar 10 Kindern.
Ein Problem könnte sein, dass es einige Buchungen gibt, die eine ungewöhnlich hohe Anzahl von Kindern angeben, wie z.B. 9 oder 10 Kinder. Dies könnte auf Fehler oder falsche Angaben hinweisen.

#### Verteilung von 'no_of_weekend_nights':

Die Mehrheit der Buchungen hat entweder keine Wochenendnächte oder 1 oder 2 Wochenendnächte.
Es gibt einige Buchungen mit 3, 4, 5, 6 und sogar 7 Wochenendnächten.

#### Verteilung von 'no_of_week_nights':

Die Mehrheit der Buchungen hat entweder 1 oder 2 Wochennächte.
Es gibt jedoch auch Buchungen mit bis zu 17 Wochennächten.


#### Verteilung von 'lead_time':

Die Verteilung der Vorlaufzeit (Zeit zwischen Buchung und Ankunft) ist ziemlich breit gestreut.
Es gibt Buchungen mit sehr kurzer Vorlaufzeit (0 bis 5 Tage) sowie Buchungen mit sehr langer Vorlaufzeit (mehrere Monate).
Hier ist der skew nach rechts sehr hoch


#### Verteilung von 'no_of_previous_cancellations':

Die Mehrheit der Buchungen hatte keine vorherigen Stornierungen.
Es gibt jedoch auch Buchungen mit 1, 2, 3, 4, 5 und sogar mehreren vorherigen Stornierungen.

#### Verteilung von 'avg_price_per_room':

Die Mehrheit der Buchungen ist um 100 verteilt.
Hier können wir eine Normalverteilung beobachten.
Zudem gibt es Buchungen für den Preis 0, das kann bedeuten, dass falsche Daten vorhanden sind oder per Aktionscode oder Gutschein gebucht wurden.

#### Verteilung von 'no_of_special_requests':

Ab einer Anzahl von 2 speziellen Anforderungen gibt es keine Stornierungen


## 3.1.3 Visualisation of feature to Target Variable

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(30, 24))

for i, feature in enumerate(features):
    row = i // 3
    col = i % 3
    sns.histplot(numeric_features[clean_df.booking_status == 1][feature],
                 bins=100,
                 color='green',
                 label='not canceled',
                 kde=True,
                 stat='density',
                 kde_kws=dict(cut=3),
                 alpha=0.4,
                 linewidth=0.25,
                 ax=axs[row, col])
    sns.histplot(numeric_features[clean_df.booking_status == 0][feature],
                 bins=100,
                 color='red',
                 label='canceled',
                 kde=True,
                 stat='density',
                 kde_kws=dict(cut=3),
                 alpha=0.4,
                 linewidth=0.25,
                 ax=axs[row, col])
    axs[row, col].set_title(f'Verteilung von {feature}')
    axs[row, col].legend()

plt.tight_layout()
plt.show()



## 3.1.4 Interpretation of the features to target variable

- Vorlaufzeit (lead time): Je länger die Vorlaufzeit, desto wahrscheinlicher ist die Stornierung der Buchung.
- Durchschnittspreis pro Zimmer (avg_price_per_room): Es gibt eine leichte Tendenz, dass höher preisige Zimmer storniert werden
- Es ist zu erkennen dass Buchungen welche in der Winter Urlaubssaison stattfinden tendenziell seltener storniert werden

Diese Fakten basieren auf den beobachteten Daten und zeigen Zusammenhänge zwischen den Merkmalen und dem Buchungsstatus auf.

## 3.1.5 Correlation Matrix

In [ ]:
corr = numeric_features.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=0.5)
plt.show()

Es liegt nur eine negative Korrelation zu der lead_time vor 

Die Nummer der speziellen Anforderungen hat eine leichte Korrelation zum booking status

Zusätzlich korrelieren die Anzahl der gäste mit dem Preis pro Raum.
Und die repeated guest spalte mit no of previous bookings.

## 3.2 Checking categorial Data

Es wäre auch möglich den Monat oder die Anzahl der Gäste mit als Kategoriale Variablen aufzunehmen oder ähnliches.
Da dieses feste Zahlen sind. Jedoch haben diese Variablen auch eine natürliche Rangfolge weshalb diese auch unter numerischen Variablen aufgeführt werden können.

In [ ]:
meal_plan_comparison = pd.crosstab(clean_df['type_of_meal_plan'], clean_df['booking_status'])
plt.figure(figsize=(10, 6))
meal_plan_comparison.plot.bar(rot=0)
plt.xlabel('type_of_meal_plan')
plt.ylabel('Anzahl')
plt.legend(['Booking Status = canceled', 'Booking Status = not canceled'])
plt.title('Type_of_meal_plan pro booking_status')
plt.show()

Die Stornierungen sind bei Meal Plan 2 deutlich höher als sonst

In [ ]:
meal_plan_comparison = pd.crosstab(clean_df['room_type_reserved'], clean_df['booking_status'])
plt.figure(figsize=(10, 6))
meal_plan_comparison.plot.bar(rot=0)
plt.xlabel('room_type_reserved')
plt.ylabel('Anzahl')
plt.legend(['Booking Status = canceled', 'Booking Status = not canceled'])
plt.title('room_type_reserved pro booking_status')
plt.xticks(rotation=45)
plt.show()

man erkennt dass es kaum einen Unterschied macht welche Art von Zimmer gewählt

In [ ]:
market_segment_comparison = pd.crosstab(clean_df['market_segment_type'], clean_df['booking_status'])
plt.figure(figsize=(10, 6))
market_segment_comparison.plot.bar(rot=0)
plt.xlabel('market_segment_type')
plt.ylabel('Anzahl der Daten')
plt.legend(['Booking Status = canceled', 'Booking Status = not canceled'])
plt.title('Vergleich von market_segment_type mit Booking Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Es ist zu erkennen, dass die Stornierungsrate bei Corporate oder Complementary Buchungen deutlich geringer sind.

In [ ]:
repeated_guest_comparison = pd.crosstab(clean_df['repeated_guest'], clean_df['booking_status'])
plt.figure(figsize=(10, 6))
repeated_guest_comparison.plot.bar(rot=0)
plt.xlabel('repeated_guest')
plt.ylabel('Anzahl')
plt.legend(['Booking Status = canceled', 'Booking Status = not canceled'])
plt.title('Vergleich von repeated_guest mit Booking Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Ein wiederkehrender Gast storniert keine Buchungen

In [ ]:
required_car_parking_space_comparison = pd.crosstab(clean_df['required_car_parking_space'], clean_df['booking_status'])
plt.figure(figsize=(10, 6))
required_car_parking_space_comparison.plot.bar(rot=0)
plt.xlabel('required_car_parking_space')
plt.ylabel('Anzahl')
plt.legend(['Booking Status = canceled', 'Booking Status = not canceled'])
plt.title('Vergleich von required_car_parking_space mit Booking Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Wenn ein Parkplatz gebucht wird, erfolgt keine Stornierung

# 4 Data Preparation

## 4.1 Ausreißer entfernen
Zunächst sollen keine Ausreißer entfernt werden oder Daten normalisiert werden.

## 4.2 Feature Engineering

In [ ]:
# Gesamt Länge Feature erstellen
clean_df['total_stay'] = clean_df['no_of_weekend_nights'] + clean_df['no_of_week_nights']
# Gesamt Gäste Feature erstellen
clean_df['total_guests'] = clean_df['no_of_adults'] + clean_df['no_of_children']
# Löschen der Features 'no_of_weekend_nights', 'no_of_week_nights', 'no_of_adults', und 'no_of_children'
data = clean_df.drop(['no_of_weekend_nights', 'no_of_week_nights', 'no_of_adults', 'no_of_children'], axis=1)
data.head()

In [ ]:
new_features = ['total_stay', 'total_guests']
fig, axs = plt.subplots(1, 2, figsize=(20, 6)) 

for i, feature in enumerate(new_features):
    sns.histplot(data[data.booking_status == 1][feature],
                 bins=100,
                 color='green',
                 label='not canceled',
                 kde=True,
                 stat='density',
                 kde_kws=dict(cut=3),
                 alpha=0.4,
                 linewidth=0.25,
                 ax=axs[i])
    sns.histplot(data[data.booking_status == 0][feature],
                 bins=100,
                 color='red',
                 label='canceled',
                 kde=True,
                 stat='density',
                 kde_kws=dict(cut=3),
                 alpha=0.4,
                 linewidth=0.25,
                 ax=axs[i])
    axs[i].set_title(f'Verteilung von {feature}')
    axs[i].legend()

plt.tight_layout()
plt.show()


Je geringer die Anzahl der Nächte und Gäste, desto höher die Wahrscheinlichkeit, dass die Buchung nicht storniert wird.

## 4.3 Kategoriale Variablen auswertbar machen

In [ ]:
#one  encoding
data_encoded = pd.get_dummies(data, columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'])
data_encoded.head()

## 4.4 Trainings und Testdatensätze erstellen

In [ ]:
#separieren der Features und der Zielvariable
X = data_encoded.drop('booking_status', axis=1)
y = data_encoded['booking_status']

#splitten der Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#standardisieren der Daten
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Modelling

Logistische Regression

In [ ]:
#trainieren des logistischen regressions Modells
clf_lr = LogisticRegression(random_state=42)
clf_lr.fit(X_train_scaled, y_train)

#vorhersage der Testdaten
y_pred_lr = clf_lr.predict(X_test_scaled)

#ausgabe der Metriken
print('Classification Report:')
print(classification_report(y_test, y_pred_lr))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_lr))

Decision Tree Modell

In [ ]:
#trainieren des Decision Tree Modells
clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt.fit(X_train_scaled, y_train)

#vorhersage der Testdaten
y_pred_dt = clf_dt.predict(X_test_scaled)

#ausgabe der Metriken
print('Classification Report:')
print(classification_report(y_test, y_pred_dt))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_dt))

Random Forest

In [ ]:
#trainieren des Random Forest Modells
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

#vorhersage der Testdaten
y_pred = clf.predict(X_test_scaled)

#ausgabe der Metriken
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

K- Nearest Neighbor

In [ ]:
#trainieren des KNN Modells
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn.fit(X_train_scaled, y_train)

#vorhersage der Testdaten
y_pred_knn = clf_knn.predict(X_test_scaled)

#ausgabe der Metriken
print('Classification Report:')
print(classification_report(y_test, y_pred_knn))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_knn))

## 6. Evaluation

The training scores for each model are as follows:

Random Forest: 0.992
Logistic Regression: 0.805
Decision Tree: 0.992
K-Nearest Neighbors: 0.899
These scores represent the accuracy of each model on the training set. If we compare these scores to the test scores we obtained earlier:

Random Forest: Test score = 0.891, Training score = 0.992
Logistic Regression: Test score = 0.804, Training score = 0.805
Decision Tree: Test score = 0.860, Training score = 0.992
K-Nearest Neighbors: Test score = 0.854, Training score = 0.899
We can see that the Random Forest and Decision Tree models have significantly higher accuracy on the training set compared to the test set. This is a sign of overfitting, as these models are performing very well on the training data but not as well on new, unseen data.

On the other hand, the Logistic Regression and K-Nearest Neighbors models have similar accuracy on both the training and test sets. This suggests that these models are not overfitting and are generalizing well to new data.

In conclusion, it appears that the Random Forest and Decision Tree models may be overfitting the training data, while the Logistic Regression and K-Nearest Neighbors models are not.

### 6.1 Finding the best Hyperparameters

6.1.1 Algorithmus um die besten Hyperparameter zu finden, um die höchste accuracy zu bekommen. (auskommentiert, da es lange braucht)

In [ ]:
'''# Define the parameter grid for RandomForestClassifier
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize a RandomForestClassifier
rf = RandomForestClassifier()

# Initialize the GridSearchCV object
rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the GridSearchCV object to the data
rf_grid_search.fit(X_train, y_train)

# Print the best parameters
print('Best parameters for RandomForestClassifier: ', rf_grid_search.best_params_)

# Define the parameter grid for DecisionTreeClassifier
dt_param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize a DecisionTreeClassifier
dt = DecisionTreeClassifier()

# Initialize the GridSearchCV object
dt_grid_search = GridSearchCV(estimator=dt, param_grid=dt_param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the GridSearchCV object to the data
dt_grid_search.fit(X_train, y_train)

# Print the best parameters
print('Best parameters for DecisionTreeClassifier: ', dt_grid_search.best_params_)'''

### Ergebnisse für die besten Parameter: 


Best parameters for RandomForestClassifier:  {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Fitting 3 folds for each of 54 candidates, totalling 162 fits

Best parameters for DecisionTreeClassifier:  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}

### 6.1 Changing Hyperparameters

In [ ]:
#trainieren des Random Forest mit den besten Parametern
clf_rf_adjusted = RandomForestClassifier(bootstrap= True, n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=5, random_state=42)
clf_rf_adjusted.fit(X_train_scaled, y_train)

#berechnen der Genauigkeit auf Trainings- und Testdaten für Random Forest
train_score_rf_adjusted = clf_rf_adjusted.score(X_train_scaled, y_train)
test_score_rf_adjusted = clf_rf_adjusted.score(X_test_scaled, y_test)

#Ausgabe der Accuracy für Random Forest
print('Training score for adjusted Random Forest:', train_score_rf_adjusted)
print('Test score for adjusted Random Forest:', test_score_rf_adjusted)

#trainieren des Decision Tree mit den besten Parametern
clf_dt_adjusted = DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=1, random_state=42)
clf_dt_adjusted.fit(X_train_scaled, y_train)

#kalkulieren der Genauigkeit auf Trainings- und Testdaten für Decision Tree
train_score_dt_adjusted = clf_dt_adjusted.score(X_train_scaled, y_train)
test_score_dt_adjusted = clf_dt_adjusted.score(X_test_scaled, y_test)

#Ausgabe der Accuracy für Decision Tree
print('Training score for adjusted Decision Tree:', train_score_dt_adjusted)
print('Test score for adjusted Decision Tree:', test_score_dt_adjusted)

### 6.2 Changing Parameters to prevent Overfitting

In [ ]:
#trainieren des Random Forest mit Parametern um Overfitting zu verhindern
clf_rf_adjusted_of = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, random_state=42)
clf_rf_adjusted_of.fit(X_train_scaled, y_train)

#kalulieren der Accuracy auf Trainings- und Testdaten für Random Forest
train_score_rf_adjusted_of = clf_rf_adjusted_of.score(X_train_scaled, y_train)
test_score_rf_adjusted_of = clf_rf_adjusted_of.score(X_test_scaled, y_test)

#Ausgabe der trainings- und test Score für Random Forest
print('Training score for adjusted Random Forest:', train_score_rf_adjusted_of)
print('Test score for adjusted Random Forest:', train_score_rf_adjusted_of)

#trainieren des Decision Tree mit Parametern um Overfitting zu verhindern
clf_dt_adjusted_of = DecisionTreeClassifier(max_depth=10, min_samples_split=10, random_state=42)
clf_dt_adjusted_of.fit(X_train_scaled, y_train)

#kalulieren der Accuracy auf Trainings- und Testdaten für Decision Tree
train_score_dt_adjusted_of = clf_dt_adjusted_of.score(X_train_scaled, y_train)
test_score_dt_adjusted_of = clf_dt_adjusted_of.score(X_test_scaled, y_test)

#Ausgabe der trainings- und test Score für Decision Tree
print('Training score for adjusted Decision Tree:', train_score_dt_adjusted_of)
print('Test score for adjusted Decision Tree:', test_score_dt_adjusted_of)

### 6.3 Oversampling on Trainingsdata with best Parameter

In [ ]:
#oversampling methode definieren
oversample = RandomOverSampler(sampling_strategy='minority')

#oversampling auf Trainingsdaten anwenden
X_train_oversampled, y_train_oversampled = oversample.fit_resample(X_train_scaled, y_train)


#trainieren des Random Forest mit den besten Parametern
clf_rf_adjusted_os = RandomForestClassifier(bootstrap= True, n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=5, random_state=42)
clf_rf_adjusted_os.fit(X_train_scaled, y_train)

#trainieren des Decision Tree mit den besten Parametern
clf_dt_adjusted_os = DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=1, random_state=42)
clf_dt_adjusted_os.fit(X_train_scaled, y_train)

#trainieren mit den oversampled Trainingsdaten
clf_rf_adjusted_os.fit(X_train_oversampled, y_train_oversampled)
clf_dt_adjusted_os.fit(X_train_oversampled, y_train_oversampled)

#Accuracy berechnen
train_score_rf_oversampled = clf_rf_adjusted.score(X_train_oversampled, y_train_oversampled)
test_score_rf_oversampled = clf_rf_adjusted.score(X_test_scaled, y_test)
train_score_dt_oversampled = clf_dt_adjusted.score(X_train_oversampled, y_train_oversampled)
test_score_dt_oversampled = clf_dt_adjusted.score(X_test_scaled, y_test)

#Ausgabe des trainings- und test Score
print('Training score for oversampled Random Forest:', train_score_rf_oversampled)
print('Test score for oversampled Random Forest:', test_score_rf_oversampled)
print('Training score for oversampled Decision Tree:', train_score_dt_oversampled)
print('Test score for oversampled Decision Tree:', test_score_dt_oversampled)

### 6.4 PLotting the Learning Curves for the 3 Models

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='g')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')
    plt.legend(loc='best')
    return plt

plot_learning_curve(clf_rf_adjusted, 'Learning Curves (Random Forest - Best Params)', X_train_scaled, y_train, cv=5, n_jobs=4)
plot_learning_curve(clf_rf_adjusted_os, 'Learning Curves (Random Forest - Best Params with Oversampling)', X_train_oversampled, y_train_oversampled, cv=5, n_jobs=4)
plot_learning_curve(clf_rf_adjusted_of, 'Learning Curves (Random Forest - Best Params to Prevent Overfitting)', X_train_scaled, y_train, cv=5, n_jobs=4)

### 6.5 Plotting the confusion Matrix for the 3 Models

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predictions for the test set
y_pred_best_params = clf_rf_adjusted.predict(X_test_scaled)
y_pred_best_params_oversampling = clf_rf_adjusted_os.predict(X_test_scaled)
y_pred_best_params_prevent_overfitting = clf_rf_adjusted_of.predict(X_test_scaled)

# Confusion matrices
cm_best_params = confusion_matrix(y_test, y_pred_best_params)
cm_best_params_oversampling = confusion_matrix(y_test, y_pred_best_params_oversampling)
cm_best_params_prevent_overfitting = confusion_matrix(y_test, y_pred_best_params_prevent_overfitting)

# Plotting
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

sns.heatmap(cm_best_params, annot=True, fmt='d', ax=ax[0], cmap='Blues')
ax[0].set_title('Confusion Matrix - Best Params')
ax[0].set_ylabel('Actual')
ax[0].set_xlabel('Predicted')

sns.heatmap(cm_best_params_oversampling, annot=True, fmt='d', ax=ax[1], cmap='Blues')
ax[1].set_title('Confusion Matrix - Best Params with Oversampling')
ax[1].set_ylabel('Actual')
ax[1].set_xlabel('Predicted')

sns.heatmap(cm_best_params_prevent_overfitting, annot=True, fmt='d', ax=ax[2], cmap='Blues')
ax[2].set_title('Confusion Matrix - Best Params to Prevent Overfitting')
ax[2].set_ylabel('Actual')
ax[2].set_xlabel('Predicted')

## 6.6 Looking at the classification reports

In [ ]:
report_best_params = classification_report(y_test, y_pred_best_params)
report_best_params_oversampling = classification_report(y_test, y_pred_best_params_oversampling)
report_best_params_prevent_overfitting = classification_report(y_test, y_pred_best_params_prevent_overfitting)

print('Classification Report - Best Params:')
print(report_best_params)
print('\nClassification Report - Best Params with Oversampling:')
print(report_best_params_oversampling)
print('\nClassification Report - Best Params to Prevent Overfitting:')
print(report_best_params_prevent_overfitting)

### 6.6 Interpretation and Model Selection

Basierend auf der Learning Curve und der Konfusionsmatrix ist für das UseCase das beste Model, der Random Forrest mit den Parametern um Overfitting zu verhindern.

1. Der Trainingsscore ist niedriger als bei den anderen beiden Modellen, was darauf hindeutet, dass das Modell die Trainingsdaten nicht so genau anpasst. Dies ist zu erwarten, da die Parameter angepasst wurden, um ein Overfitting zu verhindern. Der Cross-Validation-Score liegt näher am Trainingsscore als bei den anderen beiden Modellen, was darauf hindeutet, dass das Modell besser auf unbekannte Daten verallgemeinert. Der Cross-Validation-Score steigt, wenn mehr Trainingsbeispiele hinzugefügt werden, was darauf hindeutet, dass die Leistung des Modells auf unbekannten Daten mit mehr Trainingsdaten verbessert wird.

2. Bei einer Hotelkette kann es verherend sein False Negatives zu erzeugen, denn wenn Zimmer storniert werden und neu belegt werden kann es passieren dass es Kunden gibt die keinen Platz finden obwohl Sie gebucht hätten. Deswegen passt auch das Model mit den Besten Parametern um Overfitting zu verhindern am besten da die False Negatives am geringsten sind.